In [ ]:
import yfinance as yf
from datetime import datetime
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error
import statsmodels.api as sm
import statsmodels as smt


In [ ]:
import numpy as np
from numpy import transpose, array
from numpy.linalg import inv


param0=[1.5, 1.5]

def kf_train(Y):
    Y = transpose(array(Y))
    T = Y.shape[0]
    
    # initial values
    x_init = np.array([[100], [0.01]]) # two states: stock price, percentage change in stock price
    P_init = 900 * np.eye(len(x_init))  

    # 2x2 identity matrix
    Idn = np.eye(2)

    # allocate result matrices
    x_predict = np.zeros((T, 2, 1))      # prediction of state vector
    P_predict = np.zeros((T, 2, 2))  # prediction error covariance matrix
    x_update = np.zeros((T, 2, 1))       # estimation of state vector
    P_update = np.zeros((T, 2, 2))   # estimation error covariance matrix


    K = np.zeros((T, 2, 1))       # Kalman Gain
    v = np.zeros((T, 1, 1))  # error of estimate
    F = np.zeros((T, 1, 1))

    # initial x
    x_update[0] = x_init
    P_update[0] = P_init
    x_predict[0] = x_init
    P_predict[0] = P_init
    
    q= param0[0]
    r= param0[1]

    # matrices
    t = 1
    A = array([[1, t], [0, 1]])
    Q = q*array([[1, 0], [0, 1]])
    H = array([1, 0])
    H.shape = (1, 2)
    R = r*1

    for i in range(T):
        # prediction stage
        if i > 0:
            x_predict[i] = A @ x_update[i-1]
            P_predict[i] = A @ P_update[i-1] @ transpose(A) + Q

        # estimation stage
        v[i] = Y[i] - H @ A @ x_update[i-1]
        F[i] = H @ P_predict[i] @ transpose(H) + R

        K[i] = P_predict[i] @ transpose(H) @ inv((H @ P_predict[i] @ transpose(H)) + R)
        
        x_update[i] = x_predict[i] + K[i] @ v[i] 
        P_update[i] = (Idn - K[i] @ H) @ P_predict[i]
                
    "smoother"
    L = np.zeros((T,2,2))

    x_smooth = np.zeros((T, 2, 1))      
    P_smooth = np.zeros((T, 2, 2)) 

    x_smooth[T-1] = x_update[T-1]
    P_smooth[T-1] = P_update[T-1]  
    
    
    for t in range(T-1,0,-1): #range(start, stop, step)
        L[t-1] = P_update[t-1] @ transpose(A) @ inv(P_predict[t-1]) 
        x_smooth[t-1] = x_update[t-1] + L[t-1] @ (x_smooth[t]-A @ x_update[t])
        P_smooth[t-1] = P_update[t-1] + L[t-1] @ (P_smooth[t]-P_update[t]) @ transpose(L[t-1])
     

    x_update.shape=(Y.shape[0],2,)
    x_smooth.shape=(Y.shape[0],2,)
    
    #one step forward prediction
    y_pred = A @ x_smooth[T-1]
    
    return x_smooth[:, 0], y_pred[0]

In [ ]:
import scipy.stats as ss
from numpy.linalg import inv



def ar1_train(X, Y):
    invXTX = np.linalg.inv(X.transpose()@X)
    beta_hat = invXTX@X.transpose()@Y    
    y_hat = X@beta_hat
    
    return beta_hat, y_hat;

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import seaborn as sns



def EMA_train(X, Y):
    
    # OLS Method implementation yt=beta2*EMA(20)+uhatt        
    invXTX = np.linalg.inv(X.transpose()@X)
    beta_hat = invXTX@X.transpose()@Y
    
    y_hat = X@beta_hat
     
    return beta_hat, y_hat;

In [ ]:
from sklearn.svm import SVR  # package that we used


def svm_train(X_train, Y_train, X_pred):  
    """
    # This below model has better performance (RMSE as low as AR1), but running time is also very much longer
    
    svr_model = SVR(kernel='poly', C=1, gamma=10, degree=1, epsilon=.1, coef0=1)
    """
    
    
    "Acceptably higher RMSE (< 6) but much less running time:"
    
    svr_model = SVR(kernel='rbf', gamma=0.0005)
    result = svr_model.fit(X_train, Y_train)
 
    y_hat = result.predict(X_train)  
    y_pred = result.predict(X_pred)
    
    return y_hat, y_pred

In [ ]:
# Visa.reset_index(inplace=True)
# Visa

start_date = datetime(2019,3,1)
end_date = datetime(2020,3,31)
visa = yf.download('V',start_date ,end_date,index=False)

visa.reset_index(inplace=True)

start_date = datetime(2019,3,1)
end_date = datetime(2020,3,31)
JP = yf.download('JPM',start_date ,end_date,index=False)

start_date = datetime(2019,3,1)
end_date = datetime(2020,3,31)
Master_card = yf.download('MA',start_date ,end_date,index=False)

start_date = datetime(2019,3,1)
end_date = datetime(2020,3,31)
Bofa = yf.download('BAC',start_date ,end_date,index=False)

df_vjp = visa.merge(JP, how='left', on='Date')
df_vjp = df_vjp.rename(columns=lambda x: x.replace('_x', ''))
df_vjp = df_vjp[['Date','Adj Close','Adj Close_y']]
df_vjp.rename(columns={'Adj Close':'Adj_Close_visa','Adj Close_y':'Adj_Close_JP'},inplace=True)
df_vjp
df_vmc = visa.merge(Master_card, how='left', on='Date')
df_vmc = df_vmc.rename(columns=lambda x: x.replace('_x', ''))
df_vmc = df_vmc[['Date','Adj Close','Adj Close_y']]
df_vmc.rename(columns={'Adj Close':'Adj_Close_visa','Adj Close_y':'Adj_Close_Master'},inplace=True)
df_vba = visa.merge(Bofa, how='left', on='Date')
df_vba = df_vba.rename(columns=lambda x: x.replace('_x', ''))
df_vba = df_vba[['Date','Adj Close','Adj Close_y']]
df_vba.rename(columns={'Adj Close':'Adj_Close_visa','Adj Close_y':'Adj_Close_BOfA'},inplace=True)
df_vba
df_final = df_vmc.merge(df_vjp, how='left', on=['Date','Adj_Close_visa'])
df_final = df_final.merge(df_vba, how='left', on=['Date','Adj_Close_visa'])
df_final

df_final.set_index(['Date'], inplace=True)
df_final = df_final.diff()
df_final = df_final[1:]
df_final

In [ ]:
df_ff = pd.read_csv('F-F_Research_Data_Factors_daily.CSV',skiprows=4)
df_ff.rename(columns = {'Unnamed: 0':'Date'},inplace=True)
df_ff=df_ff[:-1]
df_ff['Date']=pd.to_datetime(df_ff['Date'])

df_ff.set_index(['Date'], inplace=True)
visa.set_index(['Date'], inplace=True)

result = visa.merge(df_ff, how='left', on='Date')
result = result.merge(df_final, how='left', on='Date')
result = result[1:]

result

In [ ]:
# import matplotlib.pyplot as plt


df = result
df['O-C'] = df['Open']-df['Close']
df['H-L'] = df['High']-df['Low']
df['open_tmr'] = df['Open'].shift(-1) # tomorrow's open price
df['close_tmr'] = df['Close'].shift(-1) # tomorrow's open price
df['logVol'] = np.log(df['Volume']) # log Volume for scaling
df=df.dropna(0)


# global variables

N = df.shape[0] # total num days
num_boot = 300 # total num bootstrap
T= 250 # start day
window = 200 # training period window


Y =df['open_tmr'][-(N-(T+1)):].values
Y_close = df['close_tmr'][-(N-(T+1)):].values

def get_Y():
    return Y, Y_close



#  methods

def ar1_bstr():
    yhat_ar1 = np.zeros(N-(T+1))
    for t in range(T+1, N):    
        # training data
        X_train = df[['Open','O-C','H-L','logVol']][t-window:t-1] # regression variables
        X_train = np.column_stack([np.ones((len(X_train),1)),X_train])
        Y_train = df['open_tmr'][t-window:t-1].values
        # one day prediction
        X_pred = df[['Open','O-C','H-L','logVol']][t-1:t]
        X_pred = np.column_stack([np.ones((len(X_pred),1)),X_pred])
        
        yhat_train = ar1_train(X_train, Y_train)[1]  
        res_train =  Y_train - yhat_train
        y_pred_all = np.zeros(num_boot)
        # bootstrap method: switching residuals
        for i in range(0, num_boot):
            err = np.random.choice(res_train, (window-1, ), replace=True)
            y_bstr = yhat_train + err   
            beta_bstr = ar1_train(X_train, y_bstr)[0]
            y_pred_bstr = X_pred@beta_bstr
            y_pred_all[i]=y_pred_bstr         
        y_pred_ar1 = y_pred_all.mean() # get mean of all bootstrap predictions        
        yhat_ar1[t-(T+1)]=y_pred_ar1 # do this for each time step
    rmse_ar1=np.sqrt(np.mean((Y-yhat_ar1)**2))
    return yhat_ar1, rmse_ar1


def ema_bstr():
    yhat_ema = np.zeros(N-(T+1))
    # 5 days EMA
    ema_short = df['Close'].ewm(span=5, adjust=False).mean()
    
    for t in range(T+1, N):
        X_train = ema_short[t-window:t-1]
        X_train = np.column_stack([np.ones((len(X_train),1)),X_train])        
        X_pred = ema_short[t-1:t]  
        X_pred = np.column_stack([np.ones((len(X_pred),1)),X_pred])        
        Y_train = df['open_tmr'][t-window:t-1].values
        yhat_fit = EMA_train(X_train,Y_train)[1] 
        res_fit = Y_train - yhat_fit
               
        # bootstrap method: switching residuals
        y_pred_all = np.zeros(num_boot)
        for i in range(0, num_boot):
            err = np.random.choice(res_fit, (window-1,), replace=True)
            y_bstr = yhat_fit + err           
            beta_bstr = EMA_train(X_train, y_bstr)[0]            
            y_pred_bstr = X_pred@beta_bstr
            y_pred_all[i]=y_pred_bstr            
        y_pred = y_pred_all.mean() # mean of all bootstrap predictions 
        yhat_ema[t-(T+1)]=y_pred        
    rmse_ema=np.sqrt(np.mean((Y-yhat_ema)**2))    
    return yhat_ema, rmse_ema

num_boot = 100
def kf_bstr():
    yhat_kf = np.zeros(N-(T+1))
    for t in range(T+1, N):    
        Y_train = df['open_tmr'][t-window:t-1].values
        yhat_fit = kf_train(Y_train)[0]
        res_fit = Y_train - yhat_fit
        
        # bootstrap method: switching residuals
        y_pred_all = np.zeros(num_boot)
        for i in range(0, num_boot):
            err = np.random.choice(res_fit, (window-1,), replace=True)
            y_bstr = yhat_fit + err                
            y_pred_bstr = kf_train(y_bstr)[1]
            y_pred_all[i]=y_pred_bstr            
        if t%100 ==0:
            print("kalman t:", t)    
        y_pred = y_pred_all.mean()
        yhat_kf[t-(T+1)]=y_pred
    
    rmse_kf=np.sqrt(np.mean((Y-yhat_kf)**2))
    return yhat_kf, rmse_kf


num_boot = 3 #  SVM is also very slow
def svm_bstr():
    yhat_svm = np.zeros(N-(T+1))
    for t in range(T+1, N):    
        X_train = df[['Close']][t-window:t-1] 
        X_train = np.column_stack([np.ones((len(X_train),1)),X_train])
        X_pred = df[['Close']][t-1:t]
        X_pred = np.column_stack([np.ones((len(X_pred),1)),X_pred])
        
        Y_train = df['open_tmr'][t-window:t-1].values    
        yhat_train = svm_train(X_train, Y_train, X_pred)[0] 
        res_train =  Y_train - yhat_train        
        y_pred_all = np.zeros(num_boot)
        # bootstrap method: switching residuals
        for i in range(0, num_boot):
            err = np.random.choice(res_train, (window-1, ), replace=True)
            y_bstr = yhat_train + err                        
            y_pred_bstr = svm_train(X_train, y_bstr, X_pred)[1]
            y_pred_all[i]=y_pred_bstr
            
        if t%100 ==0:
            print("svm t:", t)            
        y_pred_svm = y_pred_all.mean() # mean of all bootstrap predictions
        yhat_svm[t-(T+1)]=y_pred_svm # do this for each time step        
    rmse_svm=np.sqrt(np.mean((Y-yhat_svm)**2))
    return yhat_svm, rmse_svm

In [ ]:
def random_forest():
    yhat_ar1, rmse_ar1 = ar1_bstr()    
    yhat_ema, rmse_ema = ema_bstr()
    #yhat_pp, rmse_pp = pp_bstr()  # too slow 
    yhat_kf, rmse_kf = kf_bstr()
    yhat_svm, rmse_svm = svm_bstr()
    
    
    Y, Y_close = get_Y()
    T=Y.shape[0]
    
    "randome forest voting"
    Y_rf = np.zeros(T)
    for t in range(0, T):
        if min(abs(yhat_ar1[t]-Y[t]),abs(yhat_ema[t]-Y[t]),
               abs(yhat_kf[t]-Y[t]))==abs(yhat_ar1[t]-Y[t]):
            Y_rf[t] = yhat_ar1[t]
        elif min(abs(yhat_ar1[t]-Y[t]),abs(yhat_ema[t]-Y[t]),
                 abs(yhat_kf[t]-Y[t]))==abs(yhat_ema[t]-Y[t]):
            Y_rf[t] = yhat_ema[t]
        elif min(abs(yhat_ar1[t]-Y[t]),abs(yhat_ema[t]-Y[t]),
                 abs(yhat_kf[t]-Y[t]))==abs(yhat_kf[t]-Y[t]):
            Y_rf[t] = yhat_kf[t]          
        else:
            Y_rf[t] = yhat_svm[t]  
    rmse_rf=np.sqrt(np.mean((Y-Y_rf)**2))
    
    print("ar1_RMSE: ", rmse_ar1)
    print("ema_RMSE: ", rmse_ema)
    #print("PP_RMSE: ", rmse_pp)
    print("kf_RMSE: ", rmse_kf)
    print("svm_RMSE: ", rmse_svm)
    print("random forest rmse: ", rmse_rf)
    
    
    # plot 
    timevec = np.linspace(1,T,T)
    plt.figure(figsize=(30,20))
    
    ax = plt.subplot(211)
    ax.plot(timevec, Y, 'blue', label = "Y: original")
    ax.plot(timevec, yhat_ar1, 'red', label = "yhat ar1")
    ax.plot(timevec, yhat_ema, 'green', label = "yhat ema")
    #ax.plot(timevec, yhat_pp, 'pink', label = "yhat pp")
    ax.plot(timevec, yhat_svm, 'purple', label = "yhat svm")
    ax.plot(timevec, yhat_kf, 'orange', label = "yhat kf")
    plt.title('Single Model Prediction - Visa')
    ax.legend(loc=2, bbox_to_anchor=(0.5, 1.00), shadow=True, ncol=2)
    
    
    ax = plt.subplot(212)
    ax.plot(timevec, Y, 'blue', label = "Y: original")
    ax.plot(timevec, Y_rf, 'red', label = "Y_rf")
    plt.title('Random Forest Prediction - Visa')
    ax.legend(loc=2, bbox_to_anchor=(0.5, 1.00), shadow=True, ncol=2)
    plt.show()
    
    return Y, Y_rf, Y_close

In [ ]:
import scipy.stats as ss

Y, Y_rf, Y_close = random_forest()
T=Y_rf.shape[0]


#  trading rule 1: day trade"
signal_rule1 =  np.zeros(T)
for t in range(0, T):
    if Y_rf[t] > Y[t]:
        signal_rule1[t] = 1  # long signal
    elif Y_rf[t] < Y[t]:
        signal_rule1[t] = -1  # short signal
        
pos_rule1 = signal_rule1 # open and close position every day

#  trading rule 2: long short"
signal_rule2 = np.zeros(T)
pos_rule2 = np.zeros(T)
for t in range(0, T):
    if Y_rf[t] > Y[t]:
        signal_rule2[t] = 1  
    elif Y_rf[t] < Y[t]:
        signal_rule2[t] = -1  

for t in range(0, T):
    if t==0:
        pos_rule2[t] = signal_rule2[t]
    elif signal_rule2[t] != signal_rule2[t-1]: 
        # take the first long/short signal as position
        pos_rule2[t] = signal_rule2[t] 

# trading rule 3: buy hold
signal_rule3 =  np.zeros(T)
pos_rule3 = np.zeros(T)
for t in range(0, T):
    if t==0:
        signal_rule3[t] = 1  
    else:
        signal_rule3[t] =  -1

pos_rule3 = signal_rule3


#  account balances

init_bal = 1000000 # initial account balance
num_shares = 500 # number of shares being traded in one position


balance_rule1= np.zeros(T)
balance_rule2= np.zeros(T)
balance_rule3= np.zeros(T)

temp_bal1 = init_bal
temp_bal2 = init_bal
temp_bal3 = init_bal
for t in range(0,T):  
    # day trade
    balance_rule1[t] = temp_bal1 - pos_rule1[t]*num_shares*Y[t] + pos_rule1[t]*num_shares*Y_close[t]
    temp_bal1 = balance_rule1[t]
    
    # long short
    balance_rule2[t] = temp_bal2 - pos_rule2[t]*num_shares*Y[t] + pos_rule2[t]*num_shares*Y_close[t]
    temp_bal2 = balance_rule2[t]  
    
    # buy hold
    if t==0:
        # initially hold long position and not close it
        balance_rule3[t] = temp_bal3 - pos_rule3[t]*num_shares*Y[t]  
        temp_bal3 = balance_rule3[t]
    else:
        # closing initial position at time t, aka holdig it for t-1 days
        balance_rule3[t] = temp_bal3 + pos_rule3[0]*num_shares*Y_close[t] 

    
    
# plot account balance
timevec = np.linspace(1,T-1,T-1)
plt.figure(figsize=(30,20))

ax1 = plt.subplot(211)
ax1.plot(timevec, balance_rule1[1:], 'blue', label = "Day Trade")
ax1.plot(timevec, balance_rule2[1:], 'red', label = "Long Short")
ax1.plot(timevec, balance_rule3[1:], 'green', label = "Buy Hold")
ax1.legend(loc=2, bbox_to_anchor=(0.5, 1.00), shadow=True, ncol=2)
plt.title('Trading Strategy Performance - Visa')
plt.show()



# save to dataframe
combined = np.stack([Y, Y_rf, signal_rule1, pos_rule1, balance_rule1, 
                     signal_rule2, pos_rule2, balance_rule2,   
                     signal_rule3, pos_rule3, balance_rule3], axis = 1)
df = pd.DataFrame(combined, index=np.arange(0,T), 
                  columns=['Y','Y_RF','sig_1','pos_1','bal_1',
                           'sig_2','pos_2','bal_2',
                           'sig_3','pos_3','bal_3'])
df.to_csv('trades.csv')




"analytics: profit and loss, ratios, etc"

# daily profit and loss
balance_rule1 = np.insert(balance_rule1, 0, init_bal)
profit_loss1 =  np.diff(balance_rule1)

balance_rule2 = np.insert(balance_rule2, 0, init_bal)
profit_loss2 =  np.diff(balance_rule2)

balance_rule3 = np.insert(balance_rule3, 0, init_bal)
profit_loss3 =  balance_rule3 - init_bal
           

total_profit1 = sum(x for x in profit_loss1 if x>0)    
total_loss1 = abs(sum(x for x in profit_loss1 if x<0))
cnt_profit1 = sum(1 for x in profit_loss1 if x>0)
cnt_loss1 = sum(1 for x in profit_loss1 if x<0)
        

total_profit2 = sum(x for x in profit_loss2 if x>0)    
total_loss2 = abs(sum(x for x in profit_loss2 if x<0))
cnt_profit2 = sum(1 for x in profit_loss2 if x>0)
cnt_loss2 = sum(1 for x in profit_loss2 if x>0)


if profit_loss3[-1]>0:    
    total_profit3 = profit_loss3[T-1] 
    total_loss3 = 0
else:   
    total_loss3 = profit_loss3[T-1]
    total_profit3 = 0
cnt_profit3 = sum(1 for x in profit_loss3 if x>0)
cnt_loss3 = sum(1 for x in profit_loss3 if x<0)


# daily return in percentage
return1 = (balance_rule1 - init_bal)/init_bal*100
return2 = (balance_rule2 - init_bal)/init_bal*100
return3 = (balance_rule3 - init_bal)/init_bal*100


# ratios, assuming 3 month T bill rate around 2%
sharpe1 = (np.mean(return1) - 2)/np.std(return1-2)
sharpe2 = (np.mean(return2) - 2)/np.std(return2-2)
sharpe3 = (np.mean(return3) - 2)/np.std(return3-2)
sortino1 =(np.mean(return1) - 2)/np.std([x for x in return1 if x<0])
sortino2 =(np.mean(return2) - 2)/np.std([x for x in return2 if x<0])
sortino3 =(np.mean(return3) - 2)/np.std([x for x in return3 if x<0])



print("---------------------------1.DAY TRADE---------------------------")
print("number of trading days: ",T)      
print("total profit: ", total_profit1, "\t total loss: ", total_loss1, "\t profit%: ", total_profit1/init_bal)
print("net profit: ", total_profit1-total_loss1, "\t profit factor: ", total_profit1/total_loss1)
print("profit days: ", cnt_profit1, "\t loss days: ", cnt_loss1, "\t winning rate:", cnt_profit1/T)
print("aver net profit per trade: ", (total_profit1-total_loss1)/T)
print("aver daily return: ", np.mean(return1))
print("daily return std: ", np.std(return1))
print("daily return skewness: ", ss.skew(return1))
print("daily return kurtosis: ", ss.kurtosis(return1))
print("sharpe ratio: ", sharpe1, "\t sortino ratio: ",  sortino1)

den = ss.gaussian_kde(return1) 
xs = np.linspace(-3,3,400)
plt.plot(xs,den(xs))
plt.show()



print("---------------------------2.LONG SHORT---------------------------")
print("number of trading days: ",T)      
print("total profit: ", total_profit2, "\t total loss:", total_loss2, "\t profit%: ", total_profit2/init_bal)
print("net profit", total_profit2-total_loss2, "\t profit factor", total_profit2/total_loss2)
print("profit days: ", cnt_profit2, "\t loss days: ", cnt_loss2, "\t winning rate:", cnt_profit2/T)
print("aver net profit per trade: ", (total_profit2-total_loss2)/T)
print("aver daily return: ", np.mean(return2))
print("daily return std: ", np.std(return2))
print("daily return skewness: ", ss.skew(return2))
print("daily return kurtosis: ", ss.kurtosis(return2))
print("sharpe ratio: ", sharpe2, "\t sortino ratio: ",  sortino2)

den = ss.gaussian_kde(return2) 
xs = np.linspace(-3,3,400)
plt.plot(xs,den(xs))
plt.show()



print("---------------------------3.BUY HOLD---------------------------")
print("number of trading days: ",T)      
print("total profit: ", total_profit3, "\t total loss:", total_loss3, "\t profit%: ", total_profit3/init_bal)
print("net profit", total_profit3-total_loss3, "\t profit factor", total_profit3/total_loss3)
print("profit days: ", cnt_profit3, "\t loss days: ", cnt_loss3, "\t winning rate:", cnt_profit3/T)
print("aver net profit per trade: ", (total_profit3-total_loss3)/T)
print("aver daily return: ", np.mean(return3))
print("daily return std: ", np.std(return3))
print("daily return skewness: ", ss.skew(return3))
print("daily return kurtosis: ", ss.kurtosis(return3))
print("sharpe ratio: ", sharpe3, "\t sortino ratio: ",  sortino3, "--no negative return under buy hold except for initial long position")

den = ss.gaussian_kde(return3) 
xs = np.linspace(-1,6,400)
plt.plot(xs,den(xs))
plt.show()
